In [3]:
import folium
import pandas as pd
import re

In [4]:



m = folium.Map(location=[44.837789, -0.57918])

m

In [6]:
df = pd.read_csv('https://www.dropbox.com/s/7eqe4ki0mc8e832/superimmo_20211008_33.csv?dl=1')

df.location.value_counts()

Bordeaux (33000)                     1367
Mérignac (33700)                      571
Libourne (33500)                      501
Bordeaux (33800)                      487
Bordeaux (33100)                      408
                                     ... 
Saint-Martin-Lacaussade (33390)         1
Cauvignac (33690)                       1
Saint-Girons-d'Aiguevives (33920)       1
Mongauzy (33190)                        1
Guillos (33720)                         1
Name: location, Length: 479, dtype: int64

In [7]:
def getcp(location):
    # print(location)
    result = re.search(r".*\((.*)\)", location)
    # print(result)
    if result:
        return result.group(1)
    

df['Code_postal'] = df.location.apply(lambda x: getcp(str(x)))
df.info()
# ça n'a pas marché partout
df[(df['Code_postal'].isna())][['location','Code_postal','titre']]
# parce que location n'y ait pas partout ...

# on va les retirer pour le moment mais il ne faut pas qu'on les oublie ...
df = df[-(df['Code_postal'].isna())]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16567 entries, 0 to 16566
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      16567 non-null  int64  
 1   link            16567 non-null  object 
 2   ref             16404 non-null  object 
 3   agence          16567 non-null  object 
 4   description     16440 non-null  object 
 5   details         11227 non-null  object 
 6   titre           16567 non-null  object 
 7   prix_title      14085 non-null  object 
 8   prix            10982 non-null  object 
 9   taxe_fonciere   1619 non-null   object 
 10  charges         2014 non-null   object 
 11  orientation     2644 non-null   object 
 12  classe_energie  4039 non-null   object 
 13  build_date      4203 non-null   float64
 14  DPE             4428 non-null   object 
 15  surface         16369 non-null  object 
 16  piece           16388 non-null  object 
 17  chambre         13382 non-null 

In [8]:
coo=pd.read_csv('databases/laposte_hexasmal.csv',sep=";")
# print(coo)
coo.Code_postal = coo.Code_postal.astype('str')
# coo.info()
# print(coo['Code_postal'].to_list())

cpcoo={}
#for i in zip coo[['Code_postal','coordonnees_gps']].to_dict()

for cp,coo in zip (coo['Code_postal'].to_list(),coo['coordonnees_gps'].to_list()) :
    cpcoo[cp]=coo
# print(cpcoo)


def getcoo(x,cpcoo):       
    try :
        return cpcoo[x];
    except :
         return'0,0';
    

df['gps']=df.Code_postal.apply(lambda x: getcoo(str(x),cpcoo))
print(df[['Code_postal','gps']])
# print(df.set_index('Code_postal').join(coo.set_index('Code_postal')).info())



      Code_postal                             gps
0           33830   44.4861918644,-0.786911494042
1           33830   44.4861918644,-0.786911494042
2           33450    44.922024323,-0.355652161244
3           33450    44.922024323,-0.355652161244
4           33830   44.4861918644,-0.786911494042
...           ...                             ...
16562       33000    44.8572445351,-0.57369678116
16563       33230  45.0103439546,-0.0695991585994
16564       33200    44.8572445351,-0.57369678116
16565       33750    44.843131883,-0.284955496145
16566       33490   44.6132843909,-0.232960926916

[16387 rows x 2 columns]


In [9]:
df = df[(df.type=='appartement')]

df = df[-(df.prix.isna())]
df.prix = df.prix.apply(lambda x: int(float(str(x).replace('€','').replace(' ','').replace(',','.'))))
df.prix = df.prix.astype(int)

df = df[-(df.surface.isna())]
df.surface = df.surface.apply(lambda x: str(x).replace(' ','').replace(',','.'))
df.surface = df.surface.astype(float)

df['pm2']=df['prix']/df['surface']

print(df.groupby('Code_postal').describe()['pm2'])



             count          mean          std           min           25%  \
Code_postal                                                                 
33             5.0   5667.528771  1334.599915   3694.331984   4897.959184   
33000        768.0   5413.529640  1430.190621   1805.555556   4471.703980   
33100         78.0   5317.146197  4398.475550   2500.000000   4156.428571   
33110         87.0   4562.370860  1586.856220   2696.629213   3385.555360   
33112          3.0   2111.900070   124.346066   1975.000000   2058.928571   
...            ...           ...          ...           ...           ...   
33920          3.0   1911.020734   212.012643   1667.272727   1840.215311   
33950         10.0   8489.854186  3468.843193   3573.529412   6150.439342   
33970          3.0  12359.642475   807.837874  11428.571429  12102.216749   
33980         16.0   3781.080994   542.075825   2779.411765   3300.000000   
33990          1.0   2498.023256          NaN   2498.023256   2498.023256   

In [10]:
# as a folium carte
df = df[-(df.Code_postal=='33')]
tmp = df.groupby('Code_postal').agg({'pm2' : ['mean'],'gps':['min'],'location':['min']})
#tmp.columns.set_levels(['pm2_mean', 'gps', 'location'], level=1, inplace = True)
print(tmp['pm2']['mean'].describe())
for gps,location in zip (tmp['gps']['min'].to_list(),tmp['location']['min'].to_list()) :
    print(gps,location)
    


# print(tmp['pm2']['mean'])
# print(tmp)
      #
# print(df.gps.value_counts())
# qty_groupby.columns.set_levels(['max_qty', 'min_qty', 'median_qty'], level=1, inplace = True)

# for i in df.groupby('Code_postal').agg({'pm2' : ['min', 'max', 'mean'],'gps':['min'],'location':['min']}) :
#     print(i)

count       76.000000
mean      3592.050536
std       1758.488288
min        581.718750
25%       2463.454789
50%       3578.377155
75%       4129.500599
max      12359.642475
Name: mean, dtype: float64
44.8572445351,-0.57369678116 Bordeaux (33000)
44.8572445351,-0.57369678116 Bordeaux (33100)
44.8661298323,-0.602121558626 Le Bouscat (33110)
45.1404040788,-0.862340576235 Saint-Laurent-Médoc (33112)
44.6268462922,-0.746697894211 Le Barp (33114)
44.5561823053,-1.17530826791 La Teste-de-Buch - Pyla-sur-Mer (33115)
44.6529002838,-1.17429790933 Arcachon (33120)
45.0847969006,-1.04933512467 Carcans (33121)
45.5385656326,-1.07975440879 Le Verdon-sur-Mer (33123)
44.9377237217,-0.316177006649 Fronsac (33126)
44.8450267218,-0.795499117928 Martignas-sur-Jalle (33127)
44.8016009051,-0.547755768448 Bègles (33130)
44.7744744246,-0.976985580148 Lanton (33138)
44.773185218,-0.558212256384 Cadaujac (33140)
44.8548325665,-0.521018807062 Cenon (33150)
44.9333497941,-0.748518197327 Saint-Médard-en-Jalles 

In [11]:

gradient40 = ['#fafa6e',
'#faf568',
'#f9f063',
'#f9eb5d',
'#f8e658',
'#f8e153',
'#f7dc4d',
'#f7d748',
'#f6d243',
'#f6cd3f',
'#f5c83a',
'#f4c335',
'#f4bd30',
'#f3b82c',
'#f2b327',
'#f1ae23',
'#f0a91f',
'#efa41b',
'#ee9e16',
'#ed9912',
'#ec940e',
'#eb8e0a',
'#ea8906',
'#e88303',
'#e77e01',
'#e67800',
'#e47300',
'#e36d00',
'#e16700',
'#df6100',
'#de5b00',
'#dc5501',
'#da4f02',
'#d84804',
'#d64105',
'#d43a07',
'#d23209',
'#d0290b',
'#cd1e0d',
'#cb0f0f']


# for pm2 in tmp['pm2']['mean'].to_list() :
#     print(int((pm2-700)/(7000-700)*40))
#     print(gradient40[int((pm2-700)/(7000-700)*40)])
    
for gps,location,pm2 in zip (tmp['gps']['min'].to_list(),tmp['location']['min'].to_list(),tmp['pm2']['mean'].to_list()) :
    folium.CircleMarker(
    location=gps.split(','),
    radius=10,
    popup=location+" : "+str(pm2),
    color=gradient40[int((pm2-500)/(12000)*40)],
    fill=True,
    fill_color=gradient40[int((pm2-500)/(12000)*40)],
    ).add_to(m)
    
m 
    
